In [1]:
import pandas as pd 
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2,RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/aishwaryamate/Datasets/main/Pima.csv', index_col=0)
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# KBest

In [5]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
test = SelectKBest(score_func=chi2, k=5)
chi = test.fit(x,y)

In [8]:
chi.scores_

array([ 111.51969064, 1411.88704064,   17.60537322,   53.10803984,
       2175.56527292,  127.66934333,    5.39268155,  181.30368904])

In [9]:
x.columns

Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')

In [11]:
chidf = pd.DataFrame(chi.scores_).T 
chidf_colums = x.columns
chidf

,0,1,2,3,4,5,6,7
0,111.519691,1411.887041,17.605373,53.10804,2175.565273,127.669343,5.392682,181.303689


# RFE

In [12]:
lg = LogisticRegression()

rfe = RFE(lg, n_features_to_select=5)

rfe.fit(x,y)

RFE(estimator=LogisticRegression(), n_features_to_select=5)

In [13]:
rfe.ranking_

array([1, 1, 2, 3, 4, 1, 1, 1])

In [14]:
rfe.support_

array([ True,  True, False, False, False,  True,  True,  True])

In [17]:
rfe_df = pd.DataFrame(rfe.ranking_).T
rfe_df.columns = x.columns 
rfe_df

,preg,plas,pres,skin,test,mass,pedi,age
0,1,1,2,3,4,1,1,1


# Tree based approach

In [18]:
dt = DecisionTreeClassifier(criterion='entropy')

In [19]:
dt.fit(x,y)

DecisionTreeClassifier(criterion='entropy')

In [20]:
dt_df = pd.DataFrame(dt.feature_importances_).T
dt_df.columns = x.columns
dt_df

,preg,plas,pres,skin,test,mass,pedi,age
0,0.077814,0.229804,0.088008,0.02906,0.045994,0.263925,0.140252,0.125144


- Univariate selection:
    - Test, Plasma, Age, Mass
- RFE:
    - Preg, Plasma, Mass, Age
- Tree based:
    - Plasma,Mass, Pedi, Age

# Cross Validation

In [21]:
from sklearn.model_selection import KFold, LeaveOneOut, cross_val_score

In [22]:
kf = KFold(n_splits=10, random_state=1,shuffle=True)
score = cross_val_score(LogisticRegression(), x,y,cv=kf)

In [23]:
score.mean()

0.7720608339029391

# Leave One Out

In [24]:
df.shape

(768, 9)

In [25]:
le = LeaveOneOut()

le_score = cross_val_score(LogisticRegression(),x,y,cv= le)

In [29]:
le_score

array([1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1.

In [27]:
le_score.mean()

0.7786458333333334